<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3b2d670d952889477ddfbe4c271d1eab4c275bf8dc2c350673066a670a26a746
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2


    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-31 12:24:32
-------------------
qualified stocks: 88
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.37 K
Current:  1.43 C
-------------------
Today PnL: 1.08 L (0.76%)
Current PnL: -26.13 L (-16.67%)
CY Booked + Current PnL: -11.84 L (-7.55%)
-------------------
Total profit:  1.52 L
Total loss:  -27.66 L
-------------------
Total Booked + Current PnL: 15.14 L (11.66%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.0%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.51 L (64.03%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.00,0.91,-19.66,24.59,0.09,19909.0,-19812.0,80965.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.37,OX40N,NTT,DURABLES
53,NESTLEIND,1377.00,1.39,13.89,6.60,21.40,19951.0,36855.0,302281.0,5.69,68.0,X-LC,5.21,12.0,1.85,2.12,21.09,XY25,NTT,FMCG
51,MASFIN,398.61,2.58,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,56.0,H-SC,6.59,164.0,-0.05,0.68,40.65,XR,ATH,FINANCE
84,VOLTAS,1530.00,0.96,6.68,12.19,19.69,24935.0,12813.0,204555.0,-3.70,47.0,X-MC,2.13,78.0,0.51,1.43,13.93,XY25,NTT,AC
44,ITC,452.00,0.71,-1.12,12.03,10.78,28587.0,-2686.0,237632.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.06,X40,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,730.66,3.72,-1.08,48.55,46.95,60176.0,-1351.0,123946.0,-8.75,67.0,H-SC,9.47,154.0,-0.02,0.87,55.28,MH,ATH,POWER
10,BAJAJHLDNG,14451.52,3.63,2.40,26.98,30.03,36847.0,3202.0,136572.0,-0.67,52.0,X-LC,12.15,35.0,0.09,0.96,11.11,X40,ATH,FINANCE
4,ALKYLAMINE,4546.37,3.26,-31.49,182.63,93.63,126325.0,-31793.0,69170.0,-26.56,43.0,H-SC,15.74,145.0,-0.25,0.48,3.26,SR,ATH,CHEMICALS
72,SYMPHONY,1306.00,2.97,-31.95,46.91,-0.03,54631.0,-54682.0,116459.0,-45.32,60.0,M-SC,16.11,206.0,-1.00,0.81,5.83,OX40N,NTT,DURABLES
66,SHALBY,327.00,2.81,-22.70,61.83,25.10,86579.0,-41115.0,140028.0,930.30,48.0,M-SC,9.15,250.0,-0.47,0.98,12.22,XY24,NTT,HEALTHCARE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RAJESHEXPO,518.00,-4.40,-64.53,182.14,0.07,88618.0,-88523.0,48654.0,1682.83,42.0,L-SC,4.35,268.0,-1.00,0.34,21.00,OX40N,NTT,JEWELLERY
39,INDIAMART,4810.62,-1.78,-5.16,117.97,106.72,137991.0,-6365.0,116971.0,-53.11,40.0,H-SC,5.88,139.0,-0.05,0.82,17.01,AR,ATH,MISC
56,QUESS,424.00,-1.59,-30.28,104.89,42.84,47536.0,-19686.0,45320.0,-54.27,42.0,X-SC,31.62,83.0,-0.41,0.32,2.53,XY24,NTT,MISC
75,TCS,4389.97,-0.84,-11.48,36.36,20.71,121740.0,-43419.0,334818.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.88,X40,ATH,IT
47,JSWINFRA,342.00,-0.78,-3.15,20.21,16.43,43411.0,-6976.0,214798.0,-17.97,58.0,X-MC,7.64,66.0,-0.16,1.50,26.77,X40N,NTT,REALTY


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.1,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,1.39,-6.02,121.23,107.91,168211.0,-8890.0,138754.0,-23.39,42.0,M-SC,11.63,216.0,-0.05,0.97,3.46,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,1.68,-18.92,23.36,0.02,33062.0,-33025.0,141534.0,115.96,39.0,M-SC,8.35,234.0,-1.00,0.99,23.11,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,0.91,-19.66,24.59,0.09,19909.0,-19812.0,80965.0,87.83,39.0,M-SC,3.18,253.0,-1.00,0.57,6.37,OX40N,NTT,DURABLES
48,KANSAINER,340.00,1.11,-21.09,43.79,13.47,93187.0,-56862.0,212805.0,-67.53,56.0,H-SC,6.48,158.0,-0.61,1.49,8.21,XY24,NTT,PAINTS
68,SIS,528.00,-0.12,-23.48,58.46,21.26,49671.0,-26066.0,84966.0,2005.18,52.0,H-SC,4.49,166.0,-0.52,0.59,14.80,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.0,-0.22,8.14,59.3,72.27,108660.0,13789.0,183237.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.28,14.82,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
86,WIPRO,420.00,-0.22,8.14,59.30,72.27,108660.0,13789.0,183237.0,-6.99,57.0,M-LC,5.06,99.0,0.13,1.28,14.82,XR,NTT,IT
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
49,KPIGREEN,730.66,3.72,-1.08,48.55,46.95,60176.0,-1351.0,123946.0,-8.75,67.0,H-SC,9.47,154.0,-0.02,0.87,55.28,MH,ATH,POWER
51,MASFIN,398.61,2.58,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,56.0,H-SC,6.59,164.0,-0.05,0.68,40.65,XR,ATH,FINANCE
26,FINCABLES,1641.55,1.39,-6.02,121.23,107.91,168211.0,-8890.0,138754.0,-23.39,42.0,M-SC,11.63,216.0,-0.05,0.97,3.46,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.10,-19.41,92.92,55.47,182182.0,-47223.0,196063.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.46,X40N,ATH,FINANCE
23,DIXON,18931.72,1.03,-20.84,59.25,26.07,91567.0,-40677.0,154544.0,-61.53,26.0,X-MC,16.14,56.0,-0.44,1.08,1.03,X40N,ATH,IT
35,HONAUT,58357.33,-0.15,-19.09,77.14,43.32,101655.0,-31092.0,131780.0,-30.52,28.0,X-SC,7.23,90.0,-0.31,0.92,1.18,X40N,ATH,ELECTRICAL
28,GLAXO,3466.20,0.38,-3.93,44.84,39.15,85849.0,-7828.0,191456.0,-22.92,30.0,X-MC,10.87,60.0,-0.09,1.34,23.93,X40,ATH,PHARMA
3,ACC,3906.00,0.01,-27.20,125.49,64.15,217374.0,-64731.0,173220.0,-57.62,34.0,X-SC,5.66,82.0,-0.30,1.21,0.41,XY24,BTT,CEMENT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SIEMENS,4671.5,1.42,-17.45,52.04,25.51,79946.0,-32470.0,153625.0,-5.62,42.0,H-LC,0.99,49.0,-0.41,1.08,13.21,AR,ATH,ELECTRICAL
80,UNITDSPR,1644.0,0.34,4.94,15.08,20.77,37054.0,11575.0,245719.0,-4.14,51.0,X-MC,1.53,62.0,0.31,1.72,12.20,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,1.09,-5.35,27.99,21.14,60228.0,-12163.0,215176.0,-12.10,41.0,X-MC,1.65,74.0,-0.20,1.51,19.50,XY24,NTT,PAINTS
44,ITC,452.0,0.71,-1.12,12.03,10.78,28587.0,-2686.0,237632.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.06,X40,NTT,FMCG
21,DABUR,735.0,2.63,-0.54,44.49,43.70,109766.0,-1348.0,246720.0,-8.69,60.0,X-MC,2.00,72.0,-0.01,1.73,14.43,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,0.01,-27.20,125.49,64.15,217374.0,-64731.0,173220.0,-57.62,34.0,X-SC,5.66,82.0,-0.30,1.21,0.41,XY24,BTT,CEMENT
9,BAJAJHFL,181.50,-0.10,-19.41,92.92,55.47,182182.0,-47223.0,196063.0,-28.68,26.0,X-MC,16.47,64.0,-0.26,1.37,0.46,X40N,ATH,FINANCE
12,BATAINDIA,2096.00,0.53,-39.17,122.27,35.20,95700.0,-50401.0,78269.0,2.67,35.0,X-SC,17.36,92.0,-0.53,0.55,0.53,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,0.93,-10.89,43.77,28.11,78556.0,-21935.0,179475.0,-35.36,35.0,X-MC,15.20,55.0,-0.28,1.26,0.93,X40,ATH,APPARELS
23,DIXON,18931.72,1.03,-20.84,59.25,26.07,91567.0,-40677.0,154544.0,-61.53,26.0,X-MC,16.14,56.0,-0.44,1.08,1.03,X40N,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.59,-30.28,104.89,42.84,47536.0,-19686.0,45320.0,-54.27,42.0,X-SC,31.62,83.0,-0.41,0.32,2.53,XY24,NTT,MISC
60,RELAXO,1176.00,0.90,-47.68,191.09,52.31,145137.0,-69208.0,75952.0,-44.34,45.0,X-SC,7.39,91.0,-0.48,0.53,1.67,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.53,-39.17,122.27,35.20,95700.0,-50401.0,78269.0,2.67,35.0,X-SC,17.36,92.0,-0.53,0.55,0.53,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,0.74,-3.16,28.13,24.08,34583.0,-4011.0,122939.0,-10.94,42.0,X-SC,7.70,89.0,-0.12,0.86,15.84,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,-0.15,-19.09,77.14,43.32,101655.0,-31092.0,131780.0,-30.52,28.0,X-SC,7.23,90.0,-0.31,0.92,1.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-0.84,-11.48,36.36,20.71,121740.0,-43419.0,334818.0,-23.62,49.0,X-LC,3.03,1.0,-0.36,2.34,11.88,X40,ATH,IT
42,INFY,1972.00,-0.26,10.11,21.92,34.24,75516.0,31618.0,344506.0,-14.21,53.0,X-LC,7.32,2.0,0.42,2.41,19.22,X40,NTT,IT
77,TMPV,600.00,1.06,-25.07,64.11,22.98,108521.0,-56625.0,169273.0,-21.86,57.0,X-LC,5.85,3.0,-0.52,1.18,6.46,XY24,NTT,AUTO
83,VBL,671.64,0.90,-1.26,37.41,35.67,116665.0,-3988.0,311854.0,-12.96,62.0,X-LC,2.91,4.0,-0.03,2.18,12.42,X40N,ATH,FMCG
44,ITC,452.00,0.71,-1.12,12.03,10.78,28587.0,-2686.0,237632.0,-41.15,50.0,X-LC,1.99,5.0,-0.09,1.66,4.06,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,1.84,-36.73,110.04,32.89,53753.0,-28358.0,48849.0,-703.29,58.0,L-MC,6.64,259.0,-0.53,0.34,34.53,XR,NTT,BANKS
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
51,MASFIN,398.61,2.58,-1.24,23.58,22.05,22817.0,-1215.0,96765.0,-14.63,56.0,H-SC,6.59,164.0,-0.05,0.68,40.65,XR,ATH,FINANCE
14,BSOFT,831.70,1.51,-22.04,93.42,50.79,102435.0,-31003.0,109650.0,0.12,53.0,H-SC,11.11,151.0,-0.30,0.77,27.67,XR,ATH,IT
49,KPIGREEN,730.66,3.72,-1.08,48.55,46.95,60176.0,-1351.0,123946.0,-8.75,67.0,H-SC,9.47,154.0,-0.02,0.87,55.28,MH,ATH,POWER


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.21,-0.86,83.25,81.67,77423.0,-809.0,93001.0,8377.78,66.0,L-SC,16.01,271.0,-0.01,0.65,80.10,XR,NTT,CERAMICS
49,KPIGREEN,730.66,3.72,-1.08,48.55,46.95,60176.0,-1351.0,123946.0,-8.75,67.0,H-SC,9.47,154.0,-0.02,0.87,55.28,MH,ATH,POWER
14,BSOFT,831.70,1.51,-22.04,93.42,50.79,102435.0,-31003.0,109650.0,0.12,53.0,H-SC,11.11,151.0,-0.30,0.77,27.67,XR,ATH,IT
76,TITAGARH,1548.00,2.44,-15.27,72.01,45.74,147757.0,-36988.0,205189.0,12.78,66.0,H-SC,4.82,174.0,-0.25,1.44,33.45,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.00,1.82,10.34,18.48,30.73,40171.0,20372.0,217377.0,-15.42,69.0,X-MC,4.29,75.0,0.51,1.52,24.43,X40,ATH,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.95
1,25,44.12
2,50,75.11


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.20
MC    30.17
LC    25.64
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.45
X40      23.50
X40N     12.60
XY25      9.97
XR        9.65
AR        8.63
OX40N     7.36
SR        0.98
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.73
H-SC    23.59
X-LC    22.25
M-SC    11.28
X-SC     7.84
H-MC     4.71
L-SC     1.49
M-MC     1.39
M-LC     1.28
H-LC     1.08
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.14,-6.51,40.76
IT,12.50,-17.64,76.79
FINANCE,10.47,-15.46,64.47
MISC,7.00,-29.88,83.01
ELECTRICAL,5.84,-12.68,53.95
PAINTS,5.71,-13.34,30.17
INSURANCE,4.68,-1.41,36.11
PHARMA,3.96,-3.81,36.49
AUTO,2.81,-30.35,75.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3142171.0,21
AR,1339435.0,10
XR,1283851.0,13
X40,1040063.0,15
X40N,883730.0,9
OX40N,739283.0,10
XY25,382050.0,7
SR,280319.0,2
MH,60176.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3608324.0,25
M-SC,1446874.0,15
X-MC,1436673.0,16
X-LC,884965.0,13
X-SC,799100.0,8
H-MC,405399.0,3
L-SC,248062.0,3
M-LC,108660.0,1
H-LC,79946.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1240703.0      6
           AR         917126.0      5
M-SC       XY24       817166.0      6
H-SC       XR         782950.0      7
X-MC       X40        487621.0      7
           XY24       404927.0      3
X-LC       X40        369064.0      6
X-MC       X40N       354214.0      4
H-SC       OX40N      327050.0      4
M-SC       OX40N      323615.0      5
X-SC       X40N       316229.0      3
           XY24       299493.0      3
H-SC       SR         280319.0      2
H-MC       AR         215314.0      2
X-LC       X40N       213287.0      2
H-MC       XY24       190085.0      1
X-MC       XY25       189911.0      2
X-LC       XY24       189797.0      2
X-SC       X40        183378.0      2
M-SC       XR         179044.0      2
L-SC       XR         159444.0      2
M-SC       AR         127049.0      2
X-LC       XY25       112817.0      3
M-LC       XR         108660.0      1
L-SC       OX40N       88618.0      1
H-LC       AR          79946.0      1
H-SC       MH          60176.0      1
L-MC       XR          53753.0      1
M-MC       XY25        47190.0      1
L-LC       XY25        32132.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
